In [ ]:
import sys,platform
import os
import re
import docx

class docxTopaper:
    def __init__(self,pth,*who):
        self.pth=pth             
        self.who=who
        self.sysname=platform.version()
        print(self.sysname)

    def lsdir(self):
        dirs=[]
        for file in os.listdir(self.pth):
            fn=os.path.join(self.pth,file)
            if os.path.isfile(fn) and file[-4:].lower()=='docx':      
                dirs.append(fn)   
        self.dirs=dirs
        return dirs
    
    def docxTotxt(self,f_docx):
        doc=docx.Document(f_docx)
        p=doc.paragraphs
        txt=[]
        for p in doc.paragraphs:
            txt.append(p.text)
        return txt
    
    def txtTodict(self,txt):            
        dict_paper=dict()        
        for i,item in enumerate(txt):
            dict_paper[i]=item
            
        return dict_paper    

    def txt_split(self,txt):
        qNum_ptn=re.compile(r'^\d.')  #题号，以“数字. ”开头，如“1.”，“22.”等
        s_ptn=re.compile(r'.*单项选择题.*')
        m_ptn=re.compile(r'.*多项选择题.*')
        cal_ptn=re.compile(r'.*计算题.*')
        asr_ptn=re.compile(r'.*答案与解析.*')   
        asr_ptn_2=re.compile(r'.*答案部分.*') 
        
       
        mark=[]
        mark_s=[]
        mark_q=[]
        mark_m=[]
        mark_cal=[]
        mark_asr=[]
        _mark=[]
        n=0
        for t in txt:
            if qNum_ptn.match(t):
                mark_q.append(n)
            if s_ptn.match(t):
                mark_s.append(n)
            if m_ptn.match(t):
                mark_m.append(n)
            if cal_ptn.match(t):
                mark_cal.append(n)
            if asr_ptn.match(t):
                mark_asr.append(n)   
            n+=1
        
        sec_0=[mark_s[0],mark_m[0]]
        sec_1=[mark_m[0],mark_cal[0]]
        sec_2=[mark_s[1],mark_m[1]]
        sec_3=[mark_m[1],mark_cal[1]]
        
        mark.append(sec_0)
        mark.append(sec_1)
        mark.append(sec_2)
        mark.append(sec_3)
        
        mark.append(mark_q)
        
        self.mark=mark
            
        return mark
    
    def sort_qa(self,mark):        
        q=mark[4]
        new_order=[]
        num=[]
        
        #单选题
        _sec_q_s=self.mark[0]
        n=0        
        sec_q_s=[]
        while n<len(q):
            if int(q[n])>=int(_sec_q_s[0]) and int(q[n])<int(_sec_q_s[1]): 
                sec_q_s.append([q[n],q[n+1]]) 
            n+=1
        sec_q_s[-1][1]=_sec_q_s[1] #替换最后一个值
#         print(sec_q_s,len(sec_q_s))
        

        #单选题答案
        _sec_a_s=self.mark[2]  
        n=0        
        sec_a_s=[]
        while n<len(q):
            if int(q[n])>=int(_sec_a_s[0]) and int(q[n])<int(_sec_a_s[1]): 
                sec_a_s.append([q[n]+1,q[n+1]])  #q[n]为答案的序号，答案不需要序号，为q[n]+1
            n+=1
            
        sec_a_s[-1][1]=_sec_a_s[1] #替换最后一个值
#         print(sec_a_s,len(sec_a_s))
        
        
        #排列单选题目及答案
        new_s_order=[]
        if len(sec_q_s)==len(sec_a_s):
            rng=len(sec_q_s)
            print("单选题共{}题，正在生成合并序列……\n".format(rng))
            for i in range(rng):
                new_s_order.append(sec_q_s[i])
                new_s_order.append(sec_a_s[i])
        else:
            print("单选题数目与答案数目不等，请核实。\n")
            sys.exit(0)
                  
#         print(new_s_order)
        self.new_s_order=new_s_order
    
        #多选题
        _sec_q_m=self.mark[1]
        n=0        
        sec_q_m=[]
        while n<len(q):
            if int(q[n])>=int(_sec_q_m[0]) and int(q[n])<int(_sec_q_m[1]): 
                try:
                    sec_q_m.append([q[n],q[n+1]]) 
                except:
                    print("多选题目数量不匹配")
            n+=1
        sec_q_m[-1][1]=_sec_q_m[1] #替换最后一个值
#         print(sec_q_m,len(sec_q_m))
    
        #多选题答案
        _sec_a_m=self.mark[3]  
        
        n=0        
        sec_a_m=[]
        while n<len(q):
#             print(n,q[n],_sec_a_m)
            if int(q[n])>=int(_sec_a_m[0]) and int(q[n])<int(_sec_a_m[1]): 
                try:
                    sec_a_m.append([q[n]+1,q[n+1]])  #q[n]为答案的序号，答案不需要序号，为q[n]+1
                except:
                    print("多选答案数量不匹配")
            n+=1
            
        sec_a_m[-1][1]=_sec_a_m[1] #替换最后一个值
#         print(sec_a_m,len(sec_a_m))
        
        
        #排列多选题目及答案
        new_m_order=[]
        
        if len(sec_q_m)==len(sec_a_m):
            rng=len(sec_q_m)
            print("多选题共{}题，正在生成合并序列……\n".format(rng))
            for i in range(rng):
                new_m_order.append(sec_q_m[i])
                new_m_order.append(sec_a_m[i])
        else:
            print("多选题数目与答案数目不等，请核实。\n")
            sys.exit(0)
    
        self.new_m_order=new_m_order
        
        new_order.append(new_s_order)
        new_order.append(new_m_order)
        new_order.append([len(sec_q_s),len(sec_q_m)]) #写入单选/多选题目数量，后面生成文件名需要

    
        return new_order
    
    def out_txt(self,fn,dict_paper,odr):        
        if "windows" in self.sysname.lower():            
            _path=fn[:fn.rfind(u"\\")]
            _name=fn[fn.rfind(u"\\")+1:-5]
        else:
            _path=fn[:fn.rfind(u"/")]
            _name=fn[fn.rfind(u"/")+1:-5]       

        
        print("*正在导出文件……")       
        

        #单选题
#         print("单选题 \n")
        q_s=docx.Document()
        n=0
        txt_s=''
        for i in odr[0]:
            for j in range(i[0],i[1]):
                q_s.add_paragraph(dict_paper[j])
#                 print(dict_paper[j])
            
            if n%2!=0: #判断是题目+答案，完成后换行
                q_s.add_paragraph("\n")
#                 print('\n')
            n+=1
            
        fn_s=os.path.join(_path,"output",_name+"_单选题{}题.docx".format(odr[2][0]))
        q_s.save(fn_s)
            
        #多选题
#         print("多选题 \n")
        q_m=docx.Document()
        n=0
        for i in odr[1]:
            for j in range(i[0],i[1]):
                q_m.add_paragraph(dict_paper[j])
#                 print(dict_paper[j])
            
            if n%2!=0:
                q_m.add_paragraph("\n")
#                 print('\n')
            n+=1
        
        fn_m=os.path.join(_path,"output",_name+"_多选题{}题.docx".format(odr[2][1]))
        q_m.save(fn_m)
        
        print("完成\n\n")        
            
    def make_dir(self):
        path='f:\py\office'
        path_output='f:\py\office\output'
        if not os.path.exists(path):
            os.mkdir(path)
        if not os.path.exists(path_output):
            os.mkdir(path_output)        
            
    def run(self):
        if self.who and self.who[0]=="lxt":
            print("团的win10 \n")
            self.make_dir()
        else:
            print("Jack's machine \n")
            
        dirs=self.lsdir()        
        for fn in dirs:
            print("*正在处理文件：{} \n".format(fn))
            txt=self.docxTotxt(fn)
            mark_txt=self.txtTodict(txt)
            oldOrder=self.txt_split(txt)
            newOrder=self.sort_qa(oldOrder)
            self.out_txt(fn,mark_txt,newOrder)
        
        
if __name__=="__main__":
    p="/home/jack/files/py/office"
    d=docxTopaper(p)
    d.run()

    